In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/project/wiki_csai.jsonl
/kaggle/input/project/open_qa.jsonl
/kaggle/input/project/dataset.csv
/kaggle/input/stackoverflow/merged_stackoverflow_18_34_Answers.csv


In [4]:
# IMPORTS
import argparse
import json
import logging
import math
import torch
import os
import time
import random
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats
import pickle


import datasets
from collections import Counter
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from torchmetrics.classification import MulticlassAccuracy, accuracy
from sklearn.metrics import classification_report, f1_score
from tqdm.auto import tqdm
from torch.utils.data import WeightedRandomSampler
import numpy as np
from datasets import Dataset, DatasetDict

import transformers
from accelerate import Accelerator
from huggingface_hub import Repository
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from collections import Counter
from transformers.utils.versions import require_version

In [7]:
!pwd

/kaggle/working


In [3]:
data_file_path = "/kaggle/input/project/dataset.csv"
output_dir="./"
seed=42

logger = logging.getLogger(__name__)
accelerator = Accelerator()
# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)

# Setup logging, we only want one process per machine to log things on the screen.
# accelerator.is_local_main_process is only True for one process per machine.
logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if seed is not None:
    set_seed(seed)

# Handle the repository creation
if accelerator.is_main_process:
    if output_dir is not None:
        os.makedirs(output_dir, exist_ok=True)
accelerator.wait_for_everyone()

df= pd.read_csv(data_file_path)
    


In [8]:
df.chatgpt_answer[0]

'To cook clams and mussels for pasta, add the shellfish to the sauce during the last 5-10 minutes of cooking (once the sauce has already been simmering for at least 30 minutes). The clams and mussels will release their Flavor into the sauce as they cook. Be sure to discard any shellfish that do not open after cooking.'

In [45]:
df= df[df["origin"]=="AskCulinary"]
df.reset_index(drop=True, inplace=True)

In [46]:
from sklearn.model_selection import train_test_split
human_answers = df.apply(lambda x: f"Question: {x['post_question']} , Answer: {x['human_answer']}" , axis=1)
chatgpt_answers = df.apply(lambda x: f"Question: {x['post_question']} , Answer: {x['chatgpt_answer']}" , axis=1)
# human_answers= df["human_answer"]
# chatgpt_answers = df["chatgpt_answer"]

processed_df= pd.DataFrame({"text": list(human_answers)+ list(chatgpt_answers), "label": [0]* len(human_answers) + [1]*len(chatgpt_answers)})

train_val_df, test_df= train_test_split(processed_df, test_size = 0.1, stratify=processed_df.label, random_state=0)
train_df, val_df = train_test_split(train_val_df, test_size = 0.1, stratify=train_val_df.label, random_state=0)

In [47]:
sequence_classification_model="roberta-base"

tokenizer = AutoTokenizer.from_pretrained(sequence_classification_model, do_lower_case=True)
if sequence_classification_model== "gpt2":
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    
    
train_encodings = tokenizer(train_df.text.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_df.text.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_df.text.tolist(), truncation=True, padding=True, max_length=512)

train_y = list(train_df['label'])
val_y = list(val_df['label'])
test_y = list(test_df['label'])


# Create PyTorch datasets
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}, torch.tensor(self.labels[idx])

    def __len__(self):
        return len(self.labels)

train_dataset = TextClassificationDataset(train_encodings, train_y)
test_dataset = TextClassificationDataset(test_encodings, test_y)
val_dataset = TextClassificationDataset(val_encodings, val_y)

# Create PyTorch data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

In [48]:
import torch
from tqdm import tqdm
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# Load the pre-trained BERT model
model = AutoModelForSequenceClassification.from_pretrained(sequence_classification_model, num_labels=2)
if sequence_classification_model=="gpt2":
      model.config.pad_token_id = model.config.eos_token_id

# Set the hyperparameters
learning_rate = 2e-5
weight_decay = 0.01
batch_size = 32
num_epochs = 1

# Create the optimizer and the learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, eps = 1e-7)
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=50, num_training_steps=total_steps)

# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(f"The device is: {device}")
model.to(device)
for epoch in range(num_epochs):
    total_train_loss = 0
    total_train_accuracy = 0
    model.train()
    pred_logits =torch.Tensor()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch[0].items()}
        labels = batch[1].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        logits = outputs.logits
        total_train_loss+=loss.item()
        softmax =torch.softmax(logits, dim=1)
        pred_logits= torch.cat([pred_logits, softmax.cpu()], dim=0)

        preds = torch.argmax(logits, axis=1)
        train_accuracy = torch.sum(preds == labels).item() / len(labels)
        total_train_accuracy += train_accuracy
    torch.save(pred_logits,f"train_roberta_{epoch}_ep_pred_tensors.pt")

    print(f"epoch: {epoch}, Training Loss: {total_train_loss/len(train_loader)},\
     Training accuracy: {total_train_accuracy / len(train_loader)}")
    
    
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    pred_class = []
    for batch in val_loader:
        with torch.no_grad():
            inputs = {key: val.to(device) for key, val in batch[0].items()}
            labels = batch[1].to(device)
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

        total_eval_loss += loss.item()
        preds = torch.argmax(logits, axis=1)
        eval_accuracy = torch.sum(preds == labels).item() / len(labels)
    #     print(eval_accuracy)
        total_eval_accuracy += eval_accuracy

    avg_eval_accuracy = total_eval_accuracy / len(val_loader)
    avg_eval_loss = total_eval_loss / len(val_loader)

    print(f"Epoch {epoch+1}:")
    print(f"  Training loss: {loss.item():.4f}")
    print(f"  Testing loss: {avg_eval_loss:.4f}")
    print(f"  Testing accuracy: {avg_eval_accuracy:.4f}")




loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68/model.sa

The device is: cuda


100%|██████████| 476/476 [11:17<00:00,  1.42s/it]


epoch: 0, Training Loss: 0.16957227854564896,     Training accuracy: 0.9180484693877551
Epoch 1:
  Training loss: 0.0962
  Testing loss: 0.0683
  Testing accuracy: 0.9775


In [49]:
model.eval()
total_eval_accuracy = 0
total_eval_loss = 0
pred_list=[]
for batch in test_loader:
    with torch.no_grad():
        inputs = {key: val.to(device) for key, val in batch[0].items()}
        labels = batch[1].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

    total_eval_loss += loss.item()
    preds = torch.argmax(logits, axis=1)
    pred_list.extend(list(preds.cpu().numpy()))
    eval_accuracy = torch.sum(preds == labels).item() / len(labels)
#     print(eval_accuracy)
    total_eval_accuracy += eval_accuracy

avg_eval_accuracy = total_eval_accuracy / len(test_loader)
avg_eval_loss = total_eval_loss / len(test_loader)

print(f"Epoch {epoch+1}:")
print(f"  Training loss: {loss.item():.4f}")
print(f"  Testing loss: {avg_eval_loss:.4f}")
print(f"  Testing accuracy: {avg_eval_accuracy:.4f}")

Epoch 1:
  Training loss: 0.3812
  Testing loss: 0.0829
  Testing accuracy: 0.9701


In [50]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_list, digits=4))

              precision    recall  f1-score   support

           0     0.9826    0.9575    0.9699       471
           1     0.9585    0.9830    0.9706       470

    accuracy                         0.9702       941
   macro avg     0.9705    0.9703    0.9702       941
weighted avg     0.9706    0.9702    0.9702       941



In [19]:
model.save_pretrained("./")
tokenizer.save_pretrained("./")

Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin
tokenizer config file saved in ./tokenizer_config.json
Special tokens file saved in ./special_tokens_map.json


('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.json',
 './merges.txt',
 './added_tokens.json',
 './tokenizer.json')

# Test on stackoverflow

In [21]:
df= pd.read_csv(data_file_path)

df= df[df["origin"]=="StackOverflow"]
df.reset_index(drop=True, inplace=True)

from sklearn.model_selection import train_test_split
# human_answers = df.apply(lambda x: f"Question: {x['post_question']} , Answer: {x['post_answer']}" , axis=1)
# chatgpt_answers = df.apply(lambda x: f"Question: {x['post_question']} , Answer: {x['chatgpt-3.5-turbo']}" , axis=1)
# human_answers= df["human_answer"]
# chatgpt_answers = df["chatgpt_answer"]

processed_df= pd.DataFrame({"text": list(human_answers)+ list(chatgpt_answers), "label": [0]* len(human_answers) + [1]*len(chatgpt_answers)})

train_val_df, test_df= train_test_split(processed_df, test_size = 0.1, stratify=processed_df.label, random_state=0)
train_df, val_df = train_test_split(train_val_df, test_size = 0.1, stratify=train_val_df.label, random_state=0)

test_encodings = tokenizer(test_df.text.tolist(), truncation=True, padding=True, max_length=512)
test_y = list(test_df['label'])
test_dataset = TextClassificationDataset(test_encodings, test_y)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


model.eval()
total_eval_accuracy = 0
total_eval_loss = 0
pred_list=[]
for batch in test_loader:
    with torch.no_grad():
        inputs = {key: val.to(device) for key, val in batch[0].items()}
        labels = batch[1].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

    total_eval_loss += loss.item()
    preds = torch.argmax(logits, axis=1)
    pred_list.extend(list(preds.cpu().numpy()))
    eval_accuracy = torch.sum(preds == labels).item() / len(labels)
#     print(eval_accuracy)
    total_eval_accuracy += eval_accuracy

avg_eval_accuracy = total_eval_accuracy / len(test_loader)
avg_eval_loss = total_eval_loss / len(test_loader)

print(f"Epoch {epoch+1}:")
print(f"  Training loss: {loss.item():.4f}")
print(f"  Testing loss: {avg_eval_loss:.4f}")
print(f"  Testing accuracy: {avg_eval_accuracy:.4f}")

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_list, digits=4))

Epoch 1:
  Training loss: 0.6224
  Testing loss: 0.4198
  Testing accuracy: 0.8484
              precision    recall  f1-score   support

           0     0.8442    0.8553    0.8497       456
           1     0.8530    0.8418    0.8473       455

    accuracy                         0.8485       911
   macro avg     0.8486    0.8485    0.8485       911
weighted avg     0.8486    0.8485    0.8485       911



# Train on open QA test on Wiki

In [5]:
import json

with open('/kaggle/input/project/open_qa.jsonl') as f:
    lines = f.readlines()

In [23]:
open_qa_df = pd.read_json('/kaggle/input/project/open_qa.jsonl', lines=True)
wiki_csai_df = pd.read_json('/kaggle/input/project/wiki_csai.jsonl', lines=True)

from sklearn.model_selection import train_test_split
human_answers= open_qa_df["human_answers"].apply(lambda x: x[0])
chatgpt_answers = open_qa_df["chatgpt_answers"].apply(lambda x: x[0])

processed_df= pd.DataFrame({"text": list(human_answers)+ list(chatgpt_answers), "label": [0]* len(human_answers) + [1]*len(chatgpt_answers)})
train_df= processed_df

human_answers= wiki_csai_df["human_answers"].apply(lambda x: x[0])
chatgpt_answers = wiki_csai_df["chatgpt_answers"].apply(lambda x: x[0])

processed_df= pd.DataFrame({"text": list(human_answers)+ list(chatgpt_answers), "label": [0]* len(human_answers) + [1]*len(chatgpt_answers)})
test_df= processed_df

print(train_df.shape)
print(test_df.shape)

(2374, 2)
(1684, 2)


In [24]:
test_df.head()

,text,label
0,Animal cognition encompasses the mental capaci...,0
1,Human intelligence is the intellectual capabil...,0
2,The Oxford English Dictionary (OED) is the fir...,0
3,Oxford University Press (OUP) is the universit...,0
4,Artificial intelligence (AI) has been used in ...,0


In [25]:
from sklearn.model_selection import train_test_split
train_df, test_df= train_test_split(train_df, test_size=0.2, stratify=train_df.label, random_state=0)

In [26]:
sequence_classification_model="roberta-base"

import torch
from tqdm import tqdm
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(sequence_classification_model, do_lower_case=True)
if sequence_classification_model== "gpt2":
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    
    
train_encodings = tokenizer(train_df.text.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_df.text.tolist(), truncation=True, padding=True, max_length=512)

train_y = list(train_df['label'])
test_y = list(test_df['label'])


# Create PyTorch datasets
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}, torch.tensor(self.labels[idx])

    def __len__(self):
        return len(self.labels)

train_dataset = TextClassificationDataset(train_encodings, train_y)
test_dataset = TextClassificationDataset(test_encodings, test_y)

# Create PyTorch data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [27]:
import torch
from tqdm import tqdm
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# Load the pre-trained BERT model
model = AutoModelForSequenceClassification.from_pretrained(sequence_classification_model, num_labels=2)
if sequence_classification_model=="gpt2":
      model.config.pad_token_id = model.config.eos_token_id

# Set the hyperparameters
learning_rate = 2e-5
weight_decay = 0.01
batch_size = 32
num_epochs = 2

# Create the optimizer and the learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, eps = 1e-7)
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=50, num_training_steps=total_steps)

# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(f"The device is: {device}")
model.to(device)
for epoch in range(num_epochs):
    total_train_loss = 0
    total_train_accuracy = 0
    model.train()
    pred_logits =torch.Tensor()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch[0].items()}
        labels = batch[1].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        logits = outputs.logits
        total_train_loss+=loss.item()
        softmax =torch.softmax(logits, dim=1)
        pred_logits= torch.cat([pred_logits, softmax.cpu()], dim=0)

        preds = torch.argmax(logits, axis=1)
        train_accuracy = torch.sum(preds == labels).item() / len(labels)
        total_train_accuracy += train_accuracy
    torch.save(pred_logits,f"train_roberta_{epoch}_ep_pred_tensors.pt")

    print(f"epoch: {epoch}, Training Loss: {total_train_loss/len(train_loader)},\
     Training accuracy: {total_train_accuracy / len(train_loader)}")
    
    




Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should pr

The device is: cuda


100%|██████████| 119/119 [02:54<00:00,  1.47s/it]


epoch: 0, Training Loss: 0.3186355187017627,     Training accuracy: 0.8629201680672269


100%|██████████| 119/119 [02:53<00:00,  1.46s/it]

epoch: 1, Training Loss: 0.02582744897945839,     Training accuracy: 0.9942226890756303


In [35]:
test_df.label.value_counts()

0    842
1    842
Name: label, dtype: int64

In [28]:
model.eval()
total_eval_accuracy = 0
total_eval_loss = 0
pred_list=[]
for batch in test_loader:
    with torch.no_grad():
        inputs = {key: val.to(device) for key, val in batch[0].items()}
        labels = batch[1].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

    total_eval_loss += loss.item()
    preds = torch.argmax(logits, axis=1)
    pred_list.extend(list(preds.cpu().numpy()))
    eval_accuracy = torch.sum(preds == labels).item() / len(labels)
#     print(eval_accuracy)
    total_eval_accuracy += eval_accuracy

avg_eval_accuracy = total_eval_accuracy / len(test_loader)
avg_eval_loss = total_eval_loss / len(test_loader)

print(f"Epoch {epoch+1}:")
print(f"  Training loss: {loss.item():.4f}")
print(f"  Testing loss: {avg_eval_loss:.4f}")
print(f"  Testing accuracy: {avg_eval_accuracy:.4f}")

Epoch 2:
  Training loss: 0.0014
  Testing loss: 0.0064
  Testing accuracy: 0.9979


In [29]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_list, digits=4))

              precision    recall  f1-score   support

           0     0.9958    1.0000    0.9979       238
           1     1.0000    0.9958    0.9979       237

    accuracy                         0.9979       475
   macro avg     0.9979    0.9979    0.9979       475
weighted avg     0.9979    0.9979    0.9979       475



# Token perplexity

In [59]:
data_file_path = "/kaggle/input/project/dataset.csv"
df= pd.read_csv(data_file_path)
df= df[df["origin"]=="AskCulinary"]
df.reset_index(drop=True, inplace=True)

from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
from tqdm import tqdm

device = "cuda"
model_id = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

def calculate_perplexity(text):
    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(input_ids)
        log_probabilities = outputs.logits
        n_tokens = log_probabilities.shape[1]
        print(log_probabilities.shape)
        avg_log_probabilities = log_probabilities.sum(dim=-1) / n_tokens
        print(avg_log_probabilities.shape)
        perplexity = torch.exp(avg_log_probabilities)
     
    return perplexity.item()

texts = list(df.human_answer)
perplexities = [calculate_perplexity(text) for text in tqdm(texts)]
print(perplexities)




  0%|          | 0/4701 [00:00<?, ?it/s]

torch.Size([1, 104, 50257])
torch.Size([1, 104])
tensor(inf, device='cuda:0')


RuntimeError: a Tensor with 104 elements cannot be converted to Scalar

# Paraphrasing


In [5]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-_fcub085
  Running command git clone --filter=blob:none --quiet https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-_fcub085
  Resolved https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to commit 720a87a1ee557d8ed8d9a021adbdd1dd5616c5f9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8610 sha256=3594b498eb3d00e8554e18d17c8ba076400fde6e4e7b78543d1f73a72b5c6cf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-ixeytjh_/wheels/e8/ee/2a/4d6a4b2a5c37f5f750e90fa79d2ad84f444fba9b050ecbbe6d
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7f511134a5bd2c6f2b4166a605395109d7f60183eef98882094ae69dbc9eed2a
  Stored 

In [ ]:
from parrot import Parrot
import torch
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=True)


paraphrased_list=[]
for phrase in tqdm(list(df["chatgpt_answer"])):
    para_phrases = parrot.augment(input_phrase=phrase,max_return_phrases = 1, use_gpu=True)
    if para_phrases is not None:
        paraphrased_list.append(para_phrases[0][0])
    else:
        paraphrased_list.append(None)





In [17]:
para_phrases = parrot.augment(input_phrase=list(df["chatgpt_answer"])[0],max_return_phrases = 2)
print(para_phrases)

Generate config GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.29.2"
}



[('To cook clams and mussels for pasta, add the shellfish to the sauce during the last 5-10 minutes of cooking (once the sauce has already been simmering for at least 30 minutes). The clams and mussels will release their Flavor into the sauce as they cook. Be sure to discard any shellfish that do not open after cooking.', 0)]


In [18]:
list(df["chatgpt_answer"])[0]

'To cook clams and mussels for pasta, add the shellfish to the sauce during the last 5-10 minutes of cooking (once the sauce has already been simmering for at least 30 minutes). The clams and mussels will release their Flavor into the sauce as they cook. Be sure to discard any shellfish that do not open after cooking.'